# Homework - Finding APIs

In this homework we're going to reverse-engineer [Airbnb](https://en.wikipedia.org/wiki/Airbnb)'s interactive tool to estimate the potential earnings by hosting.

You can read the [press release](https://news.airbnb.com/interactive-tool-estimates-potential-monthly-host-earnings-on-airbnb/) for the tool, and view the tool on [their website](https://www.airbnb.com/host/homes?room_type=ENTIRE_HOME).

## Getting started

As a reminder, you can follow along to the basic steps to do this from the [tutorial we went over during class](https://inspectelement.org/apis.html#tutorial).

### 0) Go to the website, and open the developer tools.

As a reminder, the website we'll be inspecting today is here:
https://www.airbnb.com/host/homes?room_type=ENTIRE_HOME

Visit that link, and open up the developer tools (see step 1 in the tutorial).

### 1) Use the site as intended

With the developer tools open, go to the "Network" tab.

Go back to the Airbnb website window, and search a new address. It can be any address. For example, you can check the area surrounding Columbia by entering, "Pulitzer Hall, Broadway, New York, NY, USA".

We're just doing this step to trigger the network request (API call) that requests the potential earnings for the new address.

### 2) Find the API call

After searching a new address, you should see more entries in the "Network" tab in the developer tools.

Try to find the exact API call that pulls the estimated earnings.

Hint: this is steps [3 and 4](https://inspectelement.org/apis.html#filter-requests-by-fetchxhr) in the tutorial.

When you feel like you found the API call, copy it as a curl, and convert that curl into a Python request using [curlconverter.com](https://curlconverter.com/).

Paste the request into the cell below, and run it.

In [1]:
# Note to graders: sometimes different operating systems and browsers
# result in different values here. It might not run on your computer.
# What you want to look for is a response.json() that resembles that below

import requests

headers = {
    'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    'DNT': '1',
    'X-Airbnb-Supports-Airlock-V2': 'true',
    'X-CSRF-Token': 'null',
    'X-Airbnb-API-Key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20',
    'X-Niobe-Short-Circuited': 'true',
    'dpr': '2',
    'sec-ch-ua-platform': '"Android"',
    'device-memory': '8',
    'X-Airbnb-GraphQL-Platform-Client': 'minimalist-niobe',
    'X-Client-Version': 'db5f31305552abc4c3252f210c2c47cb0bbc6026',
    'sec-ch-ua-mobile': '?1',
    'X-CSRF-Without-Token': '1',
    'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Mobile Safari/537.36',
    'x-client-request-id': '0mrnyil1t679rc0pvn2n20viqhvo',
    'viewport-width': '1573',
    'Content-Type': 'application/json',
    'Referer': 'https://www.airbnb.com/host/homes?room_type=ENTIRE_HOME',
    'ect': '4g',
    'X-Airbnb-GraphQL-Platform': 'web',
}

params = {
    'operationName': 'GetHostEstimateData',
    'locale': 'en',
    'currency': 'USD',
    'variables': '{"durationGranularity":["MONTHLY"],"source":"HOST_LANDING_PAGE","location":{"searchQuery":"Pulitzer Hall, Broadway, New York, NY, USA"},"roomTypeCategory":"ENTIRE_HOME","bedroom":2,"fetchDebugInfo":false}',
    'extensions': '{"persistedQuery":{"version":1,"sha256Hash":"0ce03f62bd7e5cdaa639675604ea67ff782d277728c7c73b150b73c286ff9929"}}',
}

response = requests.get('https://www.airbnb.com/api/v3/GetHostEstimateData', params=params, headers=headers)

In [2]:
# use this function to show the raw JSON response
# Note to graders: NOT A QUESTION
response.json()

{'data': {'presentation': {'__typename': 'RootPresentationContainer',
   'hostEstimate': {'__typename': 'HostEstimatePresentationContainer',
    'hostEstimateScreen': {'__typename': 'HostEstimateScreen',
     'header': {'__typename': 'HostEstimateHeader',
      'sections': [{'__typename': 'HostEstimateHeaderSection',
        'label': {'__typename': 'I18nText',
         'localizedString': 'Hosts in your area earn an average of*'},
        'suffix': {'__typename': 'I18nText', 'localizedString': '/ month'},
        'value': '$11,200'},
       {'__typename': 'HostEstimateHeaderSection',
        'label': {'__typename': 'I18nText', 'localizedString': 'They earn'},
        'suffix': {'__typename': 'I18nText', 'localizedString': '/ night'},
        'value': '$448'},
       {'__typename': 'HostEstimateHeaderSection',
        'label': {'__typename': 'I18nText',
         'localizedString': "They're booked"},
        'suffix': {'__typename': 'I18nText',
         'localizedString': 'nights / month'

store the JSON response (dictionary) in a `records` variable to parse.

In [3]:
# Note to graders: NOT A QUESTION
records = response.json()

here's a trick to see the keys in the dictionary.

In [4]:
# Note to graders: NOT A QUESTION
records.keys()

dict_keys(['data', 'extensions'])

If the dictionary is nested, you can go to the next key, and then list the keys, again.

In [5]:
# Note to graders: NOT A QUESTION
records['data']['presentation'].keys()

dict_keys(['__typename', 'hostEstimate'])

### 3) Parse the response
In the cell below, traverse the JSON response and get the estimated revenue per night. Store this in a variable named `estimate`.

In [6]:
estimate = records['data']['presentation']['hostEstimate']['hostEstimateScreen']['header']['sections'][1]['value']
estimate

'$448'

### 4) Recycle the API call

Next, we're going to make the API call reusable. In the cells below fill in what the URL is for the API call

In [7]:
# what is the URL of the API we found?
url = 'https://www.airbnb.com/api/v3/GetHostEstimateData'

# copy the necessary headers here
headers = {'X-Airbnb-API-Key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20'}

# copy the necessary parameters here, but replace the hard-coded address with the variable
address = "8 Stone Street, New York, NY, USA"
params = {
    'operationName': 'GetHostEstimateData',
    'locale': 'en',
    'currency': 'USD',
    'variables': '{"durationGranularity":["MONTHLY"],"source":"HOST_LANDING_PAGE","location":{"searchQuery":"' + address + '"},"roomTypeCategory":"ENTIRE_HOME","bedroom":2,"fetchDebugInfo":false}',
    'extensions': '{"persistedQuery":{"version":1,"sha256Hash":"0ce03f62bd7e5cdaa639675604ea67ff782d277728c7c73b150b73c286ff9929"}}',
}

response = requests.get(url, params=params, headers=headers)

In [8]:
response.json()

{'data': {'presentation': {'__typename': 'RootPresentationContainer',
   'hostEstimate': {'__typename': 'HostEstimatePresentationContainer',
    'hostEstimateScreen': {'__typename': 'HostEstimateScreen',
     'header': {'__typename': 'HostEstimateHeader',
      'sections': [{'__typename': 'HostEstimateHeaderSection',
        'label': {'__typename': 'I18nText',
         'localizedString': 'Hosts in your area earn an average of*'},
        'suffix': {'__typename': 'I18nText', 'localizedString': '/ month'},
        'value': '$11,462'},
       {'__typename': 'HostEstimateHeaderSection',
        'label': {'__typename': 'I18nText', 'localizedString': 'They earn'},
        'suffix': {'__typename': 'I18nText', 'localizedString': '/ night'},
        'value': '$521'},
       {'__typename': 'HostEstimateHeaderSection',
        'label': {'__typename': 'I18nText',
         'localizedString': "They're booked"},
        'suffix': {'__typename': 'I18nText',
         'localizedString': 'nights / month'

### 5) Make more API calls
Run the API call for at least two other `address`es

In [9]:
import time

In [10]:
# here are some random addresses you can use:
addresses = [
    "Pulitzer Hall, Broadway, New York, NY, USA",
    '1932 Bryant Avenue, The Bronx, NY, USA',
    '810 39th Avenue Northeast, Naples, FL, USA',
    '86 North 6th Street, Brooklyn, NY, USA'
]

In [11]:
url = 'https://www.airbnb.com/api/v3/GetHostEstimateData'

responses = []
for address in addresses:
    params = {
        'operationName': 'GetHostEstimateData',
        'locale': 'en',
        'currency': 'USD',
        'variables': '{"durationGranularity":["MONTHLY"],"source":"HOST_LANDING_PAGE","location":{"searchQuery":"' + address + '"},"roomTypeCategory":"ENTIRE_HOME","bedroom":2,"fetchDebugInfo":false}',
        'extensions': '{"persistedQuery":{"version":1,"sha256Hash":"0ce03f62bd7e5cdaa639675604ea67ff782d277728c7c73b150b73c286ff9929"}}',
    }

    response = requests.get(url, params=params, headers=headers)
    
    # check if the API call was successful by checking the status
    if response.status_code == 200:
        responses.append(response.json())
    
    # put some time between requests
    time.sleep(1) 

### 6) What are the daily estimated earnings?
Print the estimated earning per night for each address in `responses`.

In [12]:
for records in responses:
    estimate = records['data']['presentation']['hostEstimate']['hostEstimateScreen']['header']['sections'][1]['value']
    print(estimate)

$448
$268
$160
$424


### Extra Credit
1. Check different rental configurations in the number of "bedrooms".
2. Get a random sample of addresses from usps.biglocalnews.org
    - Merge socioeconomic data from the American Community Survey
    - Format the addresses to match the input in the Airbnb site.
3. Check if there are patterns in which areas are given the lowest estimates.